In [46]:
ENV["PYTHONPATH"]="/home/gridsan/aligho/.local/lib/python3.8/site-packages/PyNormaliz-2.15-py3.8-linux-x86_64.egg";
using Crystalline, MPBUtils, SymmetryBases, JLD2, PyPlot, DelimitedFiles, PyCall, LinearAlgebra
shapely_point = pyimport("shapely.geometry").Point
shapely_polygon = pyimport("shapely.geometry.polygon").Polygon
cm = 1/2.54;

In [126]:
ks_sg2 = Vector{Vector{Float64}}()
N = 50
for i in 1:N+1
for j in 1:N+1
    kx = -0.5+1*(i-1)/N 
    ky = 0.5*(j-1)/N 
    push!(ks_sg2, [kx, ky])
end
end

ks_sg10 = Vector{Vector{Float64}}()
N = 50
for i in 1:N+1
for j in 1:N+1
    kx = 0.5*(i-1)/N 
    ky = 0.5*(j-1)/N 
    push!(ks_sg10, [kx, ky])
end
end

b1 = (2/sqrt(3))*[sqrt(3)/2, 1/2]
b2 = (2/sqrt(3))*[0, 1];
v1 = [0, 0]
v2 = -1/3*b1+2/3*b2
v3 = 1/3*(b1+b2);
c = shapely_polygon([v1, v2, v3])

ks_sg16 = Vector{Vector{Float64}}()

N = 250
for i in 1:N+1
for j in 1:N+1
    kx = -1+2*(i-1)/N 
    ky = -1+2*(j-1)/N 
    c.contains(shapely_point(kx*b1+ky*b2)) || continue
    push!(ks_sg16, [kx, ky])
end
end

In [127]:
loaded_data_sg2_lightline = load("sg2_lightline.jld2")["well_separated_nodal_points"];
loaded_data_sg10_lightline = load("sg10_lightline.jld2")["well_separated_nodal_points"];
loaded_data_sg16_lightline = load("sg16_lightline.jld2")["well_separated_nodal_points"];

In [129]:
function below_light_line(sample_info::Pair{Tuple{String, Integer}, Vector{Tuple{Integer, Integer, Float64}}}, sgnum::Integer, 
        restrict_mult::Vector{Int64} = [1, 2, 3], restrict_modes::Vector{<:AbstractString}=["te", "tm"])
    mode, real_id = sample_info[1]
    (mode ∈ restrict_modes) || return 0
    Rvecs = lattice_from_mpbparams("../symeigs/input/dim2-sg$sgnum-$(real_id - 20000)-res64-$mode.sh")[1]
    Gvecs = reciprocalbasis(Rvecs)
    tot = 0
    ks = if sgnum == 2 
            ks_sg2
        elseif sgnum == 10
            ks_sg10
        elseif sgnum == 16
            ks_sg16
        end
    for x in sample_info[2]
        mult_idx, k_idx, omega = x
        (mult_idx ∈ restrict_mult) || continue
        kx, ky = ks[k_idx]
        knorm = norm(kx * Gvecs[1] + ky * Gvecs[2])
        if omega < knorm/(2*pi)
            tot += 1
        end
    end
    return tot
end;

In [130]:
filtered_sample_infos_sg2 = filter(x -> !isempty(x[2]) && any(y[1] == 2 for y in x[2]), loaded_data_sg2_lightline);
filtered_sample_infos_sg10 = filter(x -> !isempty(x[2]) && any(y[1] == 2 for y in x[2]), loaded_data_sg10_lightline);
filtered_sample_infos_sg16 = filter(x -> !isempty(x[2]) && any(y[1] == 2 for y in x[2]), loaded_data_sg16_lightline);

In [125]:
tot_sg2_te = 0
tot_sg2_tm = 0

for sample_info in filtered_sample_infos_sg2
    tot_sg2_te += below_light_line(sample_info, 2, [1, 2, 3], ["te"])
    tot_sg2_tm += below_light_line(sample_info, 2, [1, 2, 3], ["tm"])
end
println("Percentage of samples in plane group 2 with isolated nodal points below the light line, TE:\n$(tot_sg2_te / 100)")
println("Percentage of samples in plane group 2 with isolated nodal points below the light line, TM:\n$(tot_sg2_tm / 100)")

Percentage of samples in plane group 2 with isolated nodal points below the light line, TE:
5.11
Percentage of samples in plane group 2 with isolated nodal points below the light line, TM:
0.32


In [124]:
tot_sg10_te = 0
tot_sg10_tm = 0

for sample_info in filtered_sample_infos_sg10
    tot_sg10_te += below_light_line(sample_info, 10, [1, 2, 3], ["te"])
    tot_sg10_tm += below_light_line(sample_info, 10, [1, 2, 3], ["tm"])
end
println("Percentage of samples in plane group 10 with isolated nodal points below the light line, TE: \n $(tot_sg10_te / 100)")
println("Percentage of samples in plane group 10 with isolated nodal points below the light line, TM: \n $(tot_sg10_tm / 100)")

Percentage of samples in plane group 10 with isolated nodal points below the light line, TE: 
 1.21
Percentage of samples in plane group 10 with isolated nodal points below the light line, TM: 
 0.31


In [131]:
tot_sg16_te = 0
tot_sg16_tm = 0

for sample_info in filtered_sample_infos_sg10
    tot_sg16_te += below_light_line(sample_info, 16, [1, 2, 3], ["te"])
    tot_sg16_tm += below_light_line(sample_info, 16, [1, 2, 3], ["tm"])
end
println("Percentage of samples in plane group 16 with isolated nodal points below the light line, TE: \n $(tot_sg16_te / 100)")
println("Percentage of samples in plane group 16 with isolated nodal points below the light line, TM: \n $(tot_sg16_tm / 100)")

Percentage of samples in plane group 16 with isolated nodal points below the light line, TE: 
 1.42
Percentage of samples in plane group 16 with isolated nodal points below the light line, TM: 
 0.56
